In [ ]:
from dotenv import load_dotenv
load_dotenv("admin.env")

In [ ]:
import json
from shapely.geometry import shape, MultiPolygon
from couchers.db import session_scope
from couchers.models import Cluster, ClusterRole, ClusterSubscription, Node, Page, PageType, PageVersion, Thread, Email, User
from couchers.utils import to_multi, create_coordinate
from couchers.tasks import enforce_community_memberships

In [ ]:
def create_community(session, geojson, lat, lng, name, cluster_description, main_page_title, main_page_content, admin_usernames, parent_node_id):
    admins = [session.query(User).filter(User.username == username).one() for username in admin_usernames]
    geom = shape(json.loads(geojson)["features"][0]["geometry"])
    node = Node(
        geom=to_multi(geom.wkb),
        parent_node_id=parent_node_id,
    )
    session.add(node)
    cluster = Cluster(
        name=name,
        description=cluster_description,
        parent_node=node,
        is_official_cluster=True,
    )
    session.add(cluster)
    main_page = Page(
        parent_node=cluster.parent_node,
        creator_user=admins[0],
        owner_cluster=cluster,
        type=PageType.main_page,
        thread=Thread(),
    )
    session.add(main_page)
    page_version = PageVersion(
        page=main_page,
        editor_user=admins[0],
        title=main_page_title,
        content=main_page_content,
        geom=create_coordinate(lat, lng),
        address=name,
    )
    session.add(page_version)
    for admin in admins:
        cluster.cluster_subscriptions.append(
            ClusterSubscription(
                user=admin,
                role=ClusterRole.admin,
            )
        )
    session.flush()
    return node

In [ ]:
world_geojson = """{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [[[-180,-90],[180,-90],[180,90],[-180,90],[-180,-90]]]
      }
    }
  ]
}"""

In [ ]:
with session_scope() as session:
    w_id = create_community(session, world_geojson,0,0, "Global Community", "This is the Global Couchers.org Community", "Global Community", "Welcome to the Global Couchers.org community!", ["aapeli", "itsi", "nolo"], None).id
    raise Exception("Don't run this!")